In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableLambda, RunnablePassthrough, RunnableParallel # Core LCEL components

# Load environment variables from a .env file. 🌍
# This ensures your OpenAI API key is loaded securely from your environment.
load_dotenv()

True

In [2]:
# Initialize the ChatOpenAI language model. 🤖
# This model will be used for generating the joke.
model = ChatOpenAI()

# Initialize a StrOutputParser. 📄
# This parser extracts the raw string content from the LLM's response.
parser = StrOutputParser()

In [3]:
# Define a custom Python function to count words. 🐍
# This function takes a string `text` and returns the number of words in it.
# This demonstrates how you can integrate arbitrary Python logic into LangChain pipelines.
def word_count(text):
    return len(text.split())

In [4]:
# Define the prompt template for joke generation. 📝
# It takes a `topic` as input and instructs the LLM to write a joke.
prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

In [5]:
# Define the `joke_gen_chain`. ⛓️
# This is a simple sequential chain that:
# 1. Takes the initial `topic` input.
# 2. Formats it using `prompt` to create the joke prompt.
# 3. Sends the prompt to the `model` to generate the joke.
# 4. Uses the `parser` to extract the raw joke string.
# The output of this chain will be the joke text (e.g., "Why did the AI go to the doctor? Because it had a byte!").
joke_gen_chain = RunnableSequence(prompt, model, parser)

In [6]:
# Define the `parallel_chain`. 👯
# This uses `RunnableParallel` to run multiple operations concurrently on the *same input*.
# The input to `parallel_chain` will be the *output of `joke_gen_chain`* (i.e., the joke text).
# It creates a dictionary output with two keys: 'joke' and 'word_count'.
parallel_chain = RunnableParallel({
    # 'joke' key:
    # `RunnablePassthrough()` takes the input it receives (the joke text from `joke_gen_chain`)
    # and simply passes it through unchanged. So, the 'joke' key will contain the original joke text.
    'joke': RunnablePassthrough(),

    # 'word_count' key:
    # `RunnableLambda(word_count)` wraps our custom `word_count` Python function,
    # making it a Runnable. It receives the joke text as input and returns its word count.
    'word_count': RunnableLambda(word_count)
})

In [7]:
# Define the `final_chain`. 🔗
# This chain combines `joke_gen_chain` and `parallel_chain` sequentially.
# 1. `joke_gen_chain` runs first, generating the joke text.
# 2. The *output* of `joke_gen_chain` (the joke text) becomes the *input* to `parallel_chain`.
# 3. `parallel_chain` then executes its two branches concurrently using this joke text:
#    - One branch just passes the joke through.
#    - The other branch calculates the joke's word count using the `word_count` function.
# 4. The final output of `final_chain` will be a dictionary like:
#    `{'joke': '...', 'word_count': ...}`
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

# Invoke the `final_chain` with the initial topic. 🚀
# The input `{'topic':'AI'}` is fed to `joke_gen_chain` to start the process.
result = final_chain.invoke({'topic':'AI'})

In [8]:
# Format the final output string using the results from the `final_chain`. 📊
# This combines the joke text and its calculated word count into a readable format.
final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

# Print the formatted final result.
print(final_result)

Why did the AI go to therapy? 

Because it had too many unresolved issues and needed to reboot its emotional software! 
 word count - 21
